# Variuos Stuff

## Load Data and Extract Utterances

In [3]:
from datasets import load_dataset

dataset = load_dataset("multi_woz_v22")
dataset

No config specified, defaulting to: multi_woz_v22/v2.2_active_only
Reusing dataset multi_woz_v22 (/home/ilya/.cache/huggingface/datasets/multi_woz_v22/v2.2_active_only/2.2.0/7452f16a8b502e97df5c04cc4ee5436464762fa93b1ce778dd14181e79d8b51a)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['dialogue_id', 'services', 'turns'],
        num_rows: 8437
    })
    validation: Dataset({
        features: ['dialogue_id', 'services', 'turns'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['dialogue_id', 'services', 'turns'],
        num_rows: 1000
    })
})

In [4]:
import pandas as pd
import json

# data for all further experiments
n_dialogues = 15
data = dataset['validation']['turns'][:n_dialogues]

# get single list of utterances
utterances = []
speaker = []
for dia in data:
    utterances.extend(dia['utterance'])
    speaker.extend(dia['speaker'])
df = pd.DataFrame({'text': utterances})
df.to_csv('aug-data/original.csv')
json.dump(speaker, open('aug-data/speaker.json', 'w'))

# lengths of dialogues (to be able to restore them and all dialogues from augmented utterances)
rle = [len(dia['utterance']) for dia in data]
json.dump(rle, open('aug-data/rle.json', 'w'))

In [6]:
from transformers import AutoTokenizer
from visualization_utils import get_dialogue
tok = AutoTokenizer.from_pretrained('microsoft/mpnet-base')
res = tok(['\n'.join(get_dialogue(i, 'original')) for i in range(15)])

In [8]:
for seq in res['input_ids']:
    print(len(seq))

213
370
87
229
342
212
296
269
359
282
440
133
279
219
332


## `textattack` package

[textattack augment](https://github.com/QData/TextAttack#augmenting-text-textattack-augment
)

### Embedding

Заменяем слово на другое слово, эмбеддинг которого ближе всех к эмбеддингу заменяемого слова 

In [1]:
! textattack augment\
    --input-csv aug-data/original.csv\
    --output-csv aug-data/embedding1.csv\
    --input-column text\
    --recipe embedding\
    --pct-words-to-swap .1\
    --transformations-per-example 1\
    --exclude-original

2023-07-29 22:05:42.307575: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-29 22:05:43.003568: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-29 22:05:45.185052: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-29 22:05:45.185456: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GP

In [3]:
! textattack augment\
    --input-csv aug-data/original.csv\
    --output-csv aug-data/embedding3.csv\
    --input-column text\
    --recipe embedding\
    --pct-words-to-swap .3\
    --transformations-per-example 1\
    --exclude-original

2023-07-29 22:11:09.489463: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-29 22:11:10.525161: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-29 22:11:13.182493: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-29 22:11:13.182931: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GP

In [4]:
! textattack augment\
    --input-csv aug-data/original.csv\
    --output-csv aug-data/embedding5.csv\
    --input-column text\
    --recipe embedding\
    --pct-words-to-swap .5\
    --transformations-per-example 1\
    --exclude-original

2023-07-29 22:11:56.250542: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-29 22:11:56.994763: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-29 22:11:59.231507: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-29 22:11:59.231928: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GP

In [5]:
from visualization_utils import show_augmented


for name in ['embedding1', 'embedding3', 'embedding5']:
    print(name)
    show_augmented(14, name)

embedding1


[A] I <span style='color:red;font-weight:700;text-decoration:line-through;'>need </span><span style='color:green;font-weight:700;'>require </span>a train from birmingham new street to cambridge, please. 

[B] There are 133 trains <span style='color:red;font-weight:700;text-decoration:line-through;'>traveling </span><span style='color:green;font-weight:700;'>trip </span>that path. What day would you like to travel on? 

[A] I would like to leave on <span style='color:red;font-weight:700;text-decoration:line-through;'>Wednesday </span><span style='color:green;font-weight:700;'>Monday </span>sometime after 11:30. 

[B] How about the TR1670? It meets your specifications and <span style='color:red;font-weight:700;text-decoration:line-through;'>leaves </span><span style='color:green;font-weight:700;'>sheets </span>at 11:40. 

[A] How <span style='color:red;font-weight:700;text-decoration:line-through;'>long </span><span style='color:green;font-weight:700;'>longue </span>is the train ride? 

[B] The train ride is 163 <span style='color:red;font-weight:700;text-decoration:line-through;'>minutes, </span><span style='color:green;font-weight:700;'>mins, </span>does this suit your needs? 

[A] <span style='color:red;font-weight:700;text-decoration:line-through;'>How </span><span style='color:green;font-weight:700;'>Mode </span>much is the fare on that train? 

[B] It is 75.10 <span style='color:red;font-weight:700;text-decoration:line-through;'>pounds. ¶ [A] What </span><span style='color:green;font-weight:700;'>pound. </span>

<span style='color:green;font-weight:700;'>[A] Quoi </span>is the exact arrival time? I'm also looking for a place to dine in the centre of town. I'm looking for an expensive modern european <span style='color:red;font-weight:700;text-decoration:line-through;'>restaurant. </span><span style='color:green;font-weight:700;'>lunchroom. </span>

[B] Let's take <span style='color:red;font-weight:700;text-decoration:line-through;'>care </span><span style='color:green;font-weight:700;'>healthcare </span>of the train first. The train will arrive by 14:23 in Cambridge. Would you like to book a <span style='color:red;font-weight:700;text-decoration:line-through;'>ticket? </span><span style='color:green;font-weight:700;'>tickets? </span>

[A] Thank you I do not <span style='color:red;font-weight:700;text-decoration:line-through;'>need to book </span><span style='color:green;font-weight:700;'>gotta to cookbook </span>the train, but I am looking for an expensive restaurant in the centre of town. 

[B] We have many expensive places in the center of town! Is there a <span style='color:red;font-weight:700;text-decoration:line-through;'>particular </span><span style='color:green;font-weight:700;'>singular </span>type of cuisine you'd <span style='color:red;font-weight:700;text-decoration:line-through;'>like </span><span style='color:green;font-weight:700;'>fond </span>to try? 

[A] I would like modern European <span style='color:red;font-weight:700;text-decoration:line-through;'>food. </span><span style='color:green;font-weight:700;'>diet. </span>

[B] How about darrys cookhouse and <span style='color:red;font-weight:700;text-decoration:line-through;'>wine </span><span style='color:green;font-weight:700;'>vineyard </span>shop? 

[A] Yeah that should work can I please get the <span style='color:red;font-weight:700;text-decoration:line-through;'>address </span><span style='color:green;font-weight:700;'>addressing </span>post code and phone number? 

[B] They are <span style='color:red;font-weight:700;text-decoration:line-through;'>located </span><span style='color:green;font-weight:700;'>positioned </span>at 40270 King Street City Centre. Postcode is cb11ln. Phone number is 01223505015. Can I book a <span style='color:red;font-weight:700;text-decoration:line-through;'>table </span><span style='color:green;font-weight:700;'>chalkboard </span>for you? 

[A] No thank you. I will go <span style='color:red;font-weight:700;text-decoration:line-through;'>ahead </span><span style='color:green;font-weight:700;'>upcoming </span>and book myself. Thank you so much. This was all the info I <span style='color:red;font-weight:700;text-decoration:line-through;'>needed. </span><span style='color:green;font-weight:700;'>required. </span>Have a good day. 

[B] <span style='color:red;font-weight:700;text-decoration:line-through;'>We </span><span style='color:green;font-weight:700;'>Ours </span>appreciate your business. Thank for using the Cambridge Towninfo centre! Have a great day!

embedding3


[A] I <span style='color:red;font-weight:700;text-decoration:line-through;'>need </span><span style='color:green;font-weight:700;'>needs </span>a train from birmingham <span style='color:red;font-weight:700;text-decoration:line-through;'>new </span><span style='color:green;font-weight:700;'>novo </span>street to cambridge, <span style='color:red;font-weight:700;text-decoration:line-through;'>please. </span><span style='color:green;font-weight:700;'>invites. </span>

[B] There are 133 trains <span style='color:red;font-weight:700;text-decoration:line-through;'>traveling </span><span style='color:green;font-weight:700;'>trips </span>that <span style='color:red;font-weight:700;text-decoration:line-through;'>path. </span><span style='color:green;font-weight:700;'>camino. </span>What day would you <span style='color:red;font-weight:700;text-decoration:line-through;'>like to travel </span><span style='color:green;font-weight:700;'>loves to trips </span>on? 

[A] I would <span style='color:red;font-weight:700;text-decoration:line-through;'>like to leave </span><span style='color:green;font-weight:700;'>iike to letting </span>on Wednesday <span style='color:red;font-weight:700;text-decoration:line-through;'>sometime </span><span style='color:green;font-weight:700;'>occasionally </span>after 11:30. 

[B] How about the TR1670? It <span style='color:red;font-weight:700;text-decoration:line-through;'>meets </span><span style='color:green;font-weight:700;'>fulfils </span>your <span style='color:red;font-weight:700;text-decoration:line-through;'>specifications and leaves </span><span style='color:green;font-weight:700;'>hallmarks and sheets </span>at 11:40. 

[A] How <span style='color:red;font-weight:700;text-decoration:line-through;'>long </span><span style='color:green;font-weight:700;'>longue </span>is the train ride? 

[B] The <span style='color:red;font-weight:700;text-decoration:line-through;'>train </span><span style='color:green;font-weight:700;'>forming </span>ride is 163 <span style='color:red;font-weight:700;text-decoration:line-through;'>minutes, </span><span style='color:green;font-weight:700;'>mins, </span>does this suit your <span style='color:red;font-weight:700;text-decoration:line-through;'>needs? </span><span style='color:green;font-weight:700;'>require? </span>

[A] How much is the <span style='color:red;font-weight:700;text-decoration:line-through;'>fare </span><span style='color:green;font-weight:700;'>tariffs </span>on that <span style='color:red;font-weight:700;text-decoration:line-through;'>train? </span><span style='color:green;font-weight:700;'>forming? </span>

[B] It is 75.10 <span style='color:red;font-weight:700;text-decoration:line-through;'>pounds. </span><span style='color:green;font-weight:700;'>lb. </span>

[A] What is the <span style='color:red;font-weight:700;text-decoration:line-through;'>exact </span><span style='color:green;font-weight:700;'>correct </span>arrival <span style='color:red;font-weight:700;text-decoration:line-through;'>time? </span><span style='color:green;font-weight:700;'>moment? </span>I'm <span style='color:red;font-weight:700;text-decoration:line-through;'>also looking </span><span style='color:green;font-weight:700;'>similarly researching </span>for a place to <span style='color:red;font-weight:700;text-decoration:line-through;'>dine in the centre of town. </span><span style='color:green;font-weight:700;'>lunches in the centers of cities. </span>I'm <span style='color:red;font-weight:700;text-decoration:line-through;'>looking </span><span style='color:green;font-weight:700;'>quest </span>for an expensive modern european restaurant. 

[B] Let's take care of the <span style='color:red;font-weight:700;text-decoration:line-through;'>train first. </span><span style='color:green;font-weight:700;'>forming outset. </span>The <span style='color:red;font-weight:700;text-decoration:line-through;'>train </span><span style='color:green;font-weight:700;'>forming </span>will <span style='color:red;font-weight:700;text-decoration:line-through;'>arrive </span><span style='color:green;font-weight:700;'>arrived </span>by 14:23 in Cambridge. Would you <span style='color:red;font-weight:700;text-decoration:line-through;'>like to book </span><span style='color:green;font-weight:700;'>likes to cookbook </span>a ticket? 

[A] <span style='color:red;font-weight:700;text-decoration:line-through;'>Thank </span><span style='color:green;font-weight:700;'>Thanks </span>you I do not <span style='color:red;font-weight:700;text-decoration:line-through;'>need </span><span style='color:green;font-weight:700;'>required </span>to book the <span style='color:red;font-weight:700;text-decoration:line-through;'>train, </span><span style='color:green;font-weight:700;'>forming, </span>but I am <span style='color:red;font-weight:700;text-decoration:line-through;'>looking </span><span style='color:green;font-weight:700;'>quest </span>for an expensive restaurant in the <span style='color:red;font-weight:700;text-decoration:line-through;'>centre of town. </span><span style='color:green;font-weight:700;'>centering of ciudad. </span>

[B] We have many expensive <span style='color:red;font-weight:700;text-decoration:line-through;'>places in the center of town! </span><span style='color:green;font-weight:700;'>sites in the centro of municipality! </span>Is there a <span style='color:red;font-weight:700;text-decoration:line-through;'>particular </span><span style='color:green;font-weight:700;'>especial </span>type of <span style='color:red;font-weight:700;text-decoration:line-through;'>cuisine </span><span style='color:green;font-weight:700;'>cooking </span>you'd <span style='color:red;font-weight:700;text-decoration:line-through;'>like </span><span style='color:green;font-weight:700;'>likes </span>to try? 

[A] I <span style='color:red;font-weight:700;text-decoration:line-through;'>would </span><span style='color:green;font-weight:700;'>ought </span>like modern European food. 

[B] How about darrys cookhouse and <span style='color:red;font-weight:700;text-decoration:line-through;'>wine shop? ¶ [A] Yeah </span><span style='color:green;font-weight:700;'>winemaking storing? </span>

<span style='color:green;font-weight:700;'>[A] Yep </span>that should <span style='color:red;font-weight:700;text-decoration:line-through;'>work </span><span style='color:green;font-weight:700;'>collaborating </span>can I <span style='color:red;font-weight:700;text-decoration:line-through;'>please </span><span style='color:green;font-weight:700;'>invite </span>get the <span style='color:red;font-weight:700;text-decoration:line-through;'>address </span><span style='color:green;font-weight:700;'>addresses </span>post code and phone number? 

[B] They are <span style='color:red;font-weight:700;text-decoration:line-through;'>located </span><span style='color:green;font-weight:700;'>situated </span>at 40270 <span style='color:red;font-weight:700;text-decoration:line-through;'>King </span><span style='color:green;font-weight:700;'>Emperor </span>Street City <span style='color:red;font-weight:700;text-decoration:line-through;'>Centre. </span><span style='color:green;font-weight:700;'>Centres. </span>Postcode is cb11ln. <span style='color:red;font-weight:700;text-decoration:line-through;'>Phone </span><span style='color:green;font-weight:700;'>Telefono </span>number is 01223505015. Can I <span style='color:red;font-weight:700;text-decoration:line-through;'>book a table </span><span style='color:green;font-weight:700;'>ledger a tables </span>for you? 

[A] <span style='color:red;font-weight:700;text-decoration:line-through;'>No thank </span><span style='color:green;font-weight:700;'>Nos gratitude </span>you. I will go ahead and <span style='color:red;font-weight:700;text-decoration:line-through;'>book </span><span style='color:green;font-weight:700;'>cookbook </span>myself. <span style='color:red;font-weight:700;text-decoration:line-through;'>Thank </span><span style='color:green;font-weight:700;'>Thanks </span>you so much. This was all the <span style='color:red;font-weight:700;text-decoration:line-through;'>info I needed. </span><span style='color:green;font-weight:700;'>informations I required. </span>Have a <span style='color:red;font-weight:700;text-decoration:line-through;'>good </span><span style='color:green;font-weight:700;'>buena </span>day. 

[B] We <span style='color:red;font-weight:700;text-decoration:line-through;'>appreciate </span><span style='color:green;font-weight:700;'>grateful </span>your business. Thank for <span style='color:red;font-weight:700;text-decoration:line-through;'>using </span><span style='color:green;font-weight:700;'>use </span>the Cambridge Towninfo <span style='color:red;font-weight:700;text-decoration:line-through;'>centre! </span><span style='color:green;font-weight:700;'>center! </span>Have a <span style='color:red;font-weight:700;text-decoration:line-through;'>great </span><span style='color:green;font-weight:700;'>gorgeous </span>day!

embedding5


[A] I <span style='color:red;font-weight:700;text-decoration:line-through;'>need a train </span><span style='color:green;font-weight:700;'>required a forming </span>from birmingham <span style='color:red;font-weight:700;text-decoration:line-through;'>new street </span><span style='color:green;font-weight:700;'>newest thoroughfare </span>to cambridge, <span style='color:red;font-weight:700;text-decoration:line-through;'>please. </span><span style='color:green;font-weight:700;'>invites. </span>

[B] There are 133 trains <span style='color:red;font-weight:700;text-decoration:line-through;'>traveling </span><span style='color:green;font-weight:700;'>travel </span>that <span style='color:red;font-weight:700;text-decoration:line-through;'>path. What </span><span style='color:green;font-weight:700;'>trajectories. Quel </span>day <span style='color:red;font-weight:700;text-decoration:line-through;'>would you like to travel </span><span style='color:green;font-weight:700;'>ought you iike to travelling </span>on? 

[A] I <span style='color:red;font-weight:700;text-decoration:line-through;'>would like to leave </span><span style='color:green;font-weight:700;'>ought likes to walkout </span>on <span style='color:red;font-weight:700;text-decoration:line-through;'>Wednesday sometime </span><span style='color:green;font-weight:700;'>Yesterday sometimes </span>after 11:30. 

[B] <span style='color:red;font-weight:700;text-decoration:line-through;'>How </span><span style='color:green;font-weight:700;'>Mode </span>about the TR1670? It <span style='color:red;font-weight:700;text-decoration:line-through;'>meets </span><span style='color:green;font-weight:700;'>fulfils </span>your <span style='color:red;font-weight:700;text-decoration:line-through;'>specifications and leaves </span><span style='color:green;font-weight:700;'>hallmarks and departs </span>at 11:40. 

[A] <span style='color:red;font-weight:700;text-decoration:line-through;'>How long is the train </span><span style='color:green;font-weight:700;'>Mode lang is the forming </span>ride? 

[B] The <span style='color:red;font-weight:700;text-decoration:line-through;'>train </span><span style='color:green;font-weight:700;'>forming </span>ride is 163 <span style='color:red;font-weight:700;text-decoration:line-through;'>minutes, </span><span style='color:green;font-weight:700;'>mins, </span>does this suit your <span style='color:red;font-weight:700;text-decoration:line-through;'>needs? ¶ [A] How much is the fare </span><span style='color:green;font-weight:700;'>should? </span>

<span style='color:green;font-weight:700;'>[A] Mode very is the tariffs </span>on that <span style='color:red;font-weight:700;text-decoration:line-through;'>train? </span><span style='color:green;font-weight:700;'>forming? </span>

[B] It is 75.10 <span style='color:red;font-weight:700;text-decoration:line-through;'>pounds. </span><span style='color:green;font-weight:700;'>pound. </span>

[A] What is the <span style='color:red;font-weight:700;text-decoration:line-through;'>exact arrival time? </span><span style='color:green;font-weight:700;'>precision coming times? </span>I'm <span style='color:red;font-weight:700;text-decoration:line-through;'>also looking for a place to dine in the centre of town. </span><span style='color:green;font-weight:700;'>moreover searching for a placing to dinner in the centered of urban. </span>I'm <span style='color:red;font-weight:700;text-decoration:line-through;'>looking </span><span style='color:green;font-weight:700;'>quest </span>for an expensive <span style='color:red;font-weight:700;text-decoration:line-through;'>modern european restaurant. </span><span style='color:green;font-weight:700;'>fashionable europe restaurants. </span>

[B] Let's <span style='color:red;font-weight:700;text-decoration:line-through;'>take care of the train first. </span><span style='color:green;font-weight:700;'>taking caring of the forming firstly. </span>The <span style='color:red;font-weight:700;text-decoration:line-through;'>train </span><span style='color:green;font-weight:700;'>forming </span>will <span style='color:red;font-weight:700;text-decoration:line-through;'>arrive </span><span style='color:green;font-weight:700;'>come </span>by 14:23 in <span style='color:red;font-weight:700;text-decoration:line-through;'>Cambridge. Would you like to book a ticket? ¶ [A] Thank </span><span style='color:green;font-weight:700;'>Oxford. Ought you loves to ledger a airfare? </span>

<span style='color:green;font-weight:700;'>[A] Thanked </span>you I do not <span style='color:red;font-weight:700;text-decoration:line-through;'>need to book the train, </span><span style='color:green;font-weight:700;'>necessary to cookbook the forming, </span>but I am <span style='color:red;font-weight:700;text-decoration:line-through;'>looking </span><span style='color:green;font-weight:700;'>quest </span>for an <span style='color:green;font-weight:700;'>pricey lunchroom in the centers of municipal. </span>

<span style='color:green;font-weight:700;'>[B] Ours have innumerable </span>expensive <span style='color:red;font-weight:700;text-decoration:line-through;'>restaurant in the centre of town. ¶ [B] We have many expensive places in the center of town! </span><span style='color:green;font-weight:700;'>spaces in the centres of ville! </span>Is there a <span style='color:red;font-weight:700;text-decoration:line-through;'>particular type of cuisine </span><span style='color:green;font-weight:700;'>peculiar types of gastronomy </span>you'd <span style='color:red;font-weight:700;text-decoration:line-through;'>like to try? </span><span style='color:green;font-weight:700;'>adores to strive? </span>

[A] I would <span style='color:red;font-weight:700;text-decoration:line-through;'>like modern </span><span style='color:green;font-weight:700;'>loves modernity </span>European <span style='color:red;font-weight:700;text-decoration:line-through;'>food. ¶ [B] How </span><span style='color:green;font-weight:700;'>nutritional. </span>

<span style='color:green;font-weight:700;'>[B] Mode </span>about darrys cookhouse and <span style='color:red;font-weight:700;text-decoration:line-through;'>wine shop? ¶ [A] Yeah </span><span style='color:green;font-weight:700;'>vin stores? </span>

<span style='color:green;font-weight:700;'>[A] Yup </span>that should <span style='color:red;font-weight:700;text-decoration:line-through;'>work </span><span style='color:green;font-weight:700;'>worked </span>can I <span style='color:red;font-weight:700;text-decoration:line-through;'>please get the address </span><span style='color:green;font-weight:700;'>invite obtains the solve </span>post <span style='color:red;font-weight:700;text-decoration:line-through;'>code and phone </span><span style='color:green;font-weight:700;'>codes and telephone </span>number? 

[B] They are <span style='color:red;font-weight:700;text-decoration:line-through;'>located </span><span style='color:green;font-weight:700;'>positioned </span>at 40270 <span style='color:red;font-weight:700;text-decoration:line-through;'>King Street City Centre. </span><span style='color:green;font-weight:700;'>Emperor Rue Town Centres. </span>Postcode is cb11ln. <span style='color:red;font-weight:700;text-decoration:line-through;'>Phone </span><span style='color:green;font-weight:700;'>Tel </span>number is 01223505015. Can I <span style='color:red;font-weight:700;text-decoration:line-through;'>book a table </span><span style='color:green;font-weight:700;'>ledger a tableau </span>for you? 

[A] <span style='color:red;font-weight:700;text-decoration:line-through;'>No thank </span><span style='color:green;font-weight:700;'>Not thanks </span>you. I will <span style='color:red;font-weight:700;text-decoration:line-through;'>go ahead and book </span><span style='color:green;font-weight:700;'>going forthcoming and ledger </span>myself. <span style='color:red;font-weight:700;text-decoration:line-through;'>Thank </span><span style='color:green;font-weight:700;'>Thanking </span>you so <span style='color:red;font-weight:700;text-decoration:line-through;'>much. </span><span style='color:green;font-weight:700;'>very. </span>This was all the <span style='color:red;font-weight:700;text-decoration:line-through;'>info I needed. Have a good </span><span style='color:green;font-weight:700;'>informational I require. Had a alright </span>day. 

[B] We <span style='color:red;font-weight:700;text-decoration:line-through;'>appreciate </span><span style='color:green;font-weight:700;'>grateful </span>your <span style='color:red;font-weight:700;text-decoration:line-through;'>business. Thank for using </span><span style='color:green;font-weight:700;'>firms. Thanked for employs </span>the Cambridge Towninfo <span style='color:red;font-weight:700;text-decoration:line-through;'>centre! Have a great </span><span style='color:green;font-weight:700;'>centres! Has a resplendent </span>day!

Безконтекстная замена создает грамматические ошибки

### Checklist

Заменить названия, числа, удлиннить или укоротить фразы.

In [1]:
! textattack augment\
    --input-csv aug-data/original.csv\
    --output-csv aug-data/checklist1.csv\
    --input-column text\
    --recipe checklist\
    --pct-words-to-swap .1\
    --transformations-per-example 1\
    --exclude-original

2023-07-29 22:19:59.199016: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-29 22:20:00.260409: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-29 22:20:04.345714: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-29 22:20:04.346156: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GP

In [2]:
! textattack augment\
    --input-csv aug-data/original.csv\
    --output-csv aug-data/checklist3.csv\
    --input-column text\
    --recipe checklist\
    --pct-words-to-swap .3\
    --transformations-per-example 1\
    --exclude-original

2023-07-29 22:24:21.229723: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-29 22:24:21.996574: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-29 22:24:24.224325: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-29 22:24:24.224736: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GP

In [2]:
! textattack augment\
    --input-csv aug-data/original.csv\
    --output-csv aug-data/checklist5.csv\
    --input-column text\
    --recipe checklist\
    --pct-words-to-swap .5\
    --transformations-per-example 1\
    --exclude-original

2023-07-29 22:34:21.390622: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-29 22:34:22.038400: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-29 22:34:24.171847: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-29 22:34:24.172271: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GP

In [36]:
from visualization_utils import show_augmented


for name in ['embedding1', 'embedding3', 'embedding5']:
    print(f'======= {name} =======')
    show_augmented(14, name)

[A] I need a train from <span style='color:red;font-weight:700;text-decoration:line-through;'>birmingham </span><span style='color:green;font-weight:700;'>St. Cloud </span>new street to cambridge, please. 

[B] There are <span style='color:red;font-weight:700;text-decoration:line-through;'>133 </span><span style='color:green;font-weight:700;'>93 </span>trains traveling that path. What day would you like to travel on? 

[A] I would like to leave on Wednesday sometime after <span style='color:red;font-weight:700;text-decoration:line-through;'>11:30. </span><span style='color:green;font-weight:700;'>11:4. </span>

[B] How about the TR1670? It meets your specifications and leaves at <span style='color:red;font-weight:700;text-decoration:line-through;'>11:40. </span><span style='color:green;font-weight:700;'>13:40. </span>

[A] How long is the train ride? 

[B] The train ride is <span style='color:red;font-weight:700;text-decoration:line-through;'>163 </span><span style='color:green;font-weight:700;'>249 </span>minutes, does this suit your needs? 

[A] How much is the fare on that train? 

[B] It is <span style='color:red;font-weight:700;text-decoration:line-through;'>75.10 </span><span style='color:green;font-weight:700;'>75.16 </span>pounds. 

[A] What is the exact arrival time? I'm also looking for a place to dine in the centre of town. <span style='color:red;font-weight:700;text-decoration:line-through;'>I'm </span><span style='color:green;font-weight:700;'>I am </span>looking for an expensive modern european restaurant. 

[B] <span style='color:red;font-weight:700;text-decoration:line-through;'>Let's </span><span style='color:green;font-weight:700;'>Shon </span>take care of the train first. The train will arrive by <span style='color:red;font-weight:700;text-decoration:line-through;'>14:23 </span><span style='color:green;font-weight:700;'>14:37 </span>in Cambridge. Would you like to book a ticket? 

[A] Thank you I <span style='color:red;font-weight:700;text-decoration:line-through;'>do not </span><span style='color:green;font-weight:700;'>don't </span>need to book the train, but <span style='color:red;font-weight:700;text-decoration:line-through;'>I am </span><span style='color:green;font-weight:700;'>I'm </span>looking for an expensive restaurant in the centre of town. 

[B] We have many expensive places in the center of town! Is there a particular type of cuisine <span style='color:red;font-weight:700;text-decoration:line-through;'>you'd </span><span style='color:green;font-weight:700;'>you would </span>like to try? 

[A] <span style='color:red;font-weight:700;text-decoration:line-through;'>I would </span><span style='color:green;font-weight:700;'>I'd </span>like modern European food. 

[B] How about darrys cookhouse and wine shop? 

[A] Yeah that should work can I please get the address post code and phone number? 

[B] They are located at <span style='color:red;font-weight:700;text-decoration:line-through;'>40270 </span><span style='color:green;font-weight:700;'>35271 </span>King Street City Centre. Postcode is cb11ln. Phone number is <span style='color:red;font-weight:700;text-decoration:line-through;'>01223505015. </span><span style='color:green;font-weight:700;'>46547340. </span>Can I book a table for you? 

[A] No thank you. <span style='color:red;font-weight:700;text-decoration:line-through;'>I will </span><span style='color:green;font-weight:700;'>I'll </span>go ahead and book myself. Thank you so much. This was all the info I needed. Have a good day. 

[B] We appreciate your business. Thank for using the Cambridge Towninfo centre! Have a great day!

### CLARE

Это супер медленно!

Заменяем токены, вставляем, соединяем и заменяем --- с помощью MLM.

In [54]:
! textattack augment\
    --input-csv aug-data/original.csv\
    --output-csv aug-data/clare1.csv\
    --input-column text\
    --recipe clare\
    --pct-words-to-swap .1\
    --transformations-per-example 1\
    --exclude-original

2023-07-28 16:24:01.653839: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-28 16:24:02.255426: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-28 16:24:04.327086: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-28 16:24:04.327460: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GP

In [1]:
! textattack augment\
    --input-csv aug-data/original.csv\
    --output-csv aug-data/clare3.csv\
    --input-column text\
    --recipe clare\
    --pct-words-to-swap .3\
    --transformations-per-example 1\
    --exclude-original

2023-07-29 23:06:27.875263: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-29 23:06:28.639759: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-29 23:06:31.112069: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-29 23:06:31.112455: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GP

In [2]:
! textattack augment\
    --input-csv aug-data/original.csv\
    --output-csv aug-data/clare5.csv\
    --input-column text\
    --recipe clare\
    --pct-words-to-swap .5\
    --transformations-per-example 1\
    --exclude-original

2023-07-29 23:33:36.109604: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-29 23:33:36.683459: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-29 23:33:38.520308: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-29 23:33:38.520671: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GP

In [4]:
from visualization_utils import show_augmented


for name in ['clare1', 'clare3', 'clare5']:
    print(f'======= {name} =======')
    show_augmented(14, name)

======= clare =======


[A] I need a <span style='color:green;font-weight:700;'>carriage </span>train from birmingham new street to cambridge, please. 

[B] There are <span style='color:green;font-weight:700;'>about </span>133 trains traveling that path. What day would you like to travel on? 

[A] I <span style='color:green;font-weight:700;'>absolutely </span>would like to leave on Wednesday sometime after 11:30. 

[B] How about the TR1670? It meets your specifications and leaves at <span style='color:green;font-weight:700;'>the </span>11:40. 

[A] How long is <span style='color:green;font-weight:700;'>not </span>the train ride? 

[B] The train ride is <span style='color:green;font-weight:700;'>barely </span>163 minutes, does this suit your needs? 

[A] How much is the fare <span style='color:green;font-weight:700;'>calculated </span>on that train? 

[B] It is 75.10 <span style='color:green;font-weight:700;'>lb </span>pounds. 

[A] What is <span style='color:green;font-weight:700;'>often </span>the exact arrival time? I'm also looking for a place to dine in the centre of town. I'm looking for an expensive modern european restaurant. 

[B] Let's take care of the train <span style='color:red;font-weight:700;text-decoration:line-through;'>first. </span><span style='color:green;font-weight:700;'>logistics. </span>The train will arrive by 14:23 in Cambridge. Would you like to book a <span style='color:green;font-weight:700;'>tour </span>ticket? 

[A] Thank you <span style='color:green;font-weight:700;'>guys </span>I do not need to book the train, but I am looking for an expensive restaurant in the centre of town. 

[B] We have many expensive places in the center of town! Is there a particular type <span style='color:green;font-weight:700;'>fare </span>of cuisine you'd like to try? 

[A] I would like modern <span style='color:red;font-weight:700;text-decoration:line-through;'>European </span><span style='color:green;font-weight:700;'>Asian </span>food. 

[B] How about darrys cookhouse and wine <span style='color:green;font-weight:700;'>glasses </span>shop? 

[A] Yeah that should work can I please <span style='color:green;font-weight:700;'>try </span>get the address post code and phone number? 

[B] They are located at 40270 King <span style='color:red;font-weight:700;text-decoration:line-through;'>Street City </span><span style='color:green;font-weight:700;'>don </span>Centre. <span style='color:red;font-weight:700;text-decoration:line-through;'>Postcode </span><span style='color:green;font-weight:700;'>Name </span>is cb11ln. Phone number is 01223505015. Can I book a table for you? 

[A] No thank you. I will go ahead and book myself. <span style='color:green;font-weight:700;'>Seriously </span>Thank you so much. This was all the info I needed. Have a good day. 

[B] We appreciate your <span style='color:red;font-weight:700;text-decoration:line-through;'>business. </span><span style='color:green;font-weight:700;'>inconvenience. </span>Thank for using the Cambridge Towninfo centre! Have a great day!

======= clare3 =======


[A] I need a train from <span style='color:red;font-weight:700;text-decoration:line-through;'>birmingham </span><span style='color:green;font-weight:700;'>whole </span>new street to <span style='color:green;font-weight:700;'>stupid </span>cambridge, please. 

[B] There are 133 trains traveling <span style='color:green;font-weight:700;'>parallel </span>that path. What <span style='color:red;font-weight:700;text-decoration:line-through;'>day </span><span style='color:green;font-weight:700;'>paths longer </span>would you like to travel on? 

[A] <span style='color:red;font-weight:700;text-decoration:line-through;'>I </span><span style='color:green;font-weight:700;'>They </span>would like to leave on Wednesday sometime after <span style='color:green;font-weight:700;'>this </span>11:30. 

[B] How about <span style='color:green;font-weight:700;'>replacing </span>the TR1670? It meets your specifications and <span style='color:red;font-weight:700;text-decoration:line-through;'>leaves </span><span style='color:green;font-weight:700;'>start </span>at 11:40. 

[A] How long is the train <span style='color:red;font-weight:700;text-decoration:line-through;'>ride? </span><span style='color:green;font-weight:700;'>up? </span>

[B] The <span style='color:green;font-weight:700;'>shortest </span>train ride is 163 minutes, does this suit your <span style='color:green;font-weight:700;'>ride </span>needs? 

[A] How much is the <span style='color:red;font-weight:700;text-decoration:line-through;'>fare </span><span style='color:green;font-weight:700;'>pineapple </span>on that <span style='color:red;font-weight:700;text-decoration:line-through;'>train? </span><span style='color:green;font-weight:700;'>burger? </span>

[B] It is <span style='color:red;font-weight:700;text-decoration:line-through;'>75.10 </span><span style='color:green;font-weight:700;'>75.Average 10 </span>pounds. 

[A] What is the <span style='color:green;font-weight:700;'>tourists </span>exact arrival time? I'm <span style='color:green;font-weight:700;'>then </span>also looking for a place to dine in the centre <span style='color:green;font-weight:700;'>circle </span>of town. <span style='color:red;font-weight:700;text-decoration:line-through;'>I'm </span><span style='color:green;font-weight:700;'>But </span>looking for an expensive modern european <span style='color:red;font-weight:700;text-decoration:line-through;'>restaurant. </span><span style='color:green;font-weight:700;'>inn. </span>

[B] Let's take care <span style='color:red;font-weight:700;text-decoration:line-through;'>of the train first. </span><span style='color:green;font-weight:700;'>details of the train etiquette. </span>The train will arrive by 14:23 in Cambridge. Would you <span style='color:green;font-weight:700;'>even </span>like to book a <span style='color:green;font-weight:700;'>quicker </span>ticket? 

[A] Thank you I do <span style='color:green;font-weight:700;'>certainly </span>not need to book <span style='color:green;font-weight:700;'>up </span>the train, but I am looking for <span style='color:green;font-weight:700;'>only </span>an expensive restaurant in the <span style='color:red;font-weight:700;text-decoration:line-through;'>centre </span><span style='color:green;font-weight:700;'>name </span>of town. 

[B] We have many expensive places in the center <span style='color:red;font-weight:700;text-decoration:line-through;'>of </span><span style='color:green;font-weight:700;'>downtown of Boston </span>town! Is there a particular type of cuisine you'd like to <span style='color:green;font-weight:700;'>personally </span>try? 

[A] I would like <span style='color:red;font-weight:700;text-decoration:line-through;'>modern European </span><span style='color:green;font-weight:700;'>healthier </span>food. 

[B] How about darrys cookhouse and wine <span style='color:green;font-weight:700;'>bottle </span>shop? 

[A] Yeah that should work can I <span style='color:green;font-weight:700;'>ma </span>please get <span style='color:green;font-weight:700;'>thru </span>the address post code and phone number? 

[B] They are <span style='color:red;font-weight:700;text-decoration:line-through;'>located </span><span style='color:green;font-weight:700;'>currently </span>at 40270 King <span style='color:red;font-weight:700;text-decoration:line-through;'>Street City </span><span style='color:green;font-weight:700;'>Shopping </span>Centre. Postcode <span style='color:green;font-weight:700;'>numbers </span>is cb11ln. Phone number is 01223505015. Can I <span style='color:green;font-weight:700;'>actually </span>book a table for you? 

[A] No <span style='color:green;font-weight:700;'>worries </span>thank you. I will <span style='color:green;font-weight:700;'>soon </span>go ahead and book myself. Thank you so much. <span style='color:red;font-weight:700;text-decoration:line-through;'>This </span><span style='color:green;font-weight:700;'>Inside </span>was all the info I needed. Have a good <span style='color:green;font-weight:700;'>business </span>day. 

[B] We appreciate your business. <span style='color:green;font-weight:700;'>More </span>Thank for using the Cambridge Towninfo centre! <span style='color:green;font-weight:700;'>Your </span>Have a <span style='color:red;font-weight:700;text-decoration:line-through;'>great </span><span style='color:green;font-weight:700;'>decent </span>day!

======= clare5 =======


[A] I need <span style='color:red;font-weight:700;text-decoration:line-through;'>a </span><span style='color:green;font-weight:700;'>send a wooden </span>train from birmingham new <span style='color:green;font-weight:700;'>ish </span>street to cambridge, please. 

[B] There are 133 <span style='color:red;font-weight:700;text-decoration:line-through;'>trains traveling that </span><span style='color:green;font-weight:700;'>miles walking </span>path. What day <span style='color:green;font-weight:700;'>routes </span>would <span style='color:red;font-weight:700;text-decoration:line-through;'>you like to </span><span style='color:green;font-weight:700;'>take you like to do </span>travel on? 

[A] I would like <span style='color:red;font-weight:700;text-decoration:line-through;'>to </span><span style='color:green;font-weight:700;'>you to soon </span>leave on <span style='color:green;font-weight:700;'>Sheffield </span>Wednesday sometime after 11:30. 

[B] <span style='color:red;font-weight:700;text-decoration:line-through;'>How </span><span style='color:green;font-weight:700;'>Asked </span>about the TR1670? <span style='color:red;font-weight:700;text-decoration:line-through;'>It meets </span><span style='color:green;font-weight:700;'>UPS set </span>your <span style='color:red;font-weight:700;text-decoration:line-through;'>specifications </span><span style='color:green;font-weight:700;'>deadline </span>and <span style='color:green;font-weight:700;'>Santa </span>leaves at 11:40. 

[A] How <span style='color:red;font-weight:700;text-decoration:line-through;'>long </span><span style='color:green;font-weight:700;'>predictable BART </span>is the train <span style='color:red;font-weight:700;text-decoration:line-through;'>ride? </span><span style='color:green;font-weight:700;'>station? </span>

[B] The <span style='color:red;font-weight:700;text-decoration:line-through;'>train ride </span><span style='color:green;font-weight:700;'>demo format </span>is 163 minutes, does this <span style='color:green;font-weight:700;'>not </span>suit your <span style='color:red;font-weight:700;text-decoration:line-through;'>needs? </span><span style='color:green;font-weight:700;'>specifications? </span>

[A] How <span style='color:green;font-weight:700;'>else </span>much is the fare <span style='color:green;font-weight:700;'>tag </span>on that train? 

[B] It is <span style='color:red;font-weight:700;text-decoration:line-through;'>75.10 pounds. </span><span style='color:green;font-weight:700;'>75.Just 10 degrees. </span>

[A] What is <span style='color:green;font-weight:700;'>indeed </span>the exact arrival time? I'm also looking for a place to <span style='color:green;font-weight:700;'>traditionally </span>dine in <span style='color:green;font-weight:700;'>inside </span>the centre of town. I'm <span style='color:red;font-weight:700;text-decoration:line-through;'>looking for </span><span style='color:green;font-weight:700;'>going for only </span>an <span style='color:green;font-weight:700;'>uber </span>expensive modern european <span style='color:green;font-weight:700;'>neighbourhood </span>restaurant. 

[B] Let's take care <span style='color:green;font-weight:700;'>all </span>of the train first. The <span style='color:green;font-weight:700;'>passenger </span>train will arrive <span style='color:green;font-weight:700;'>tomorrow </span>by 14:23 <span style='color:green;font-weight:700;'>BST </span>in Cambridge. Would you like to book <span style='color:red;font-weight:700;text-decoration:line-through;'>a </span><span style='color:green;font-weight:700;'>on a standby </span>ticket? 

[A] Thank you I <span style='color:green;font-weight:700;'>simply </span>do not <span style='color:red;font-weight:700;text-decoration:line-through;'>need </span><span style='color:green;font-weight:700;'>HAVE </span>to book the <span style='color:green;font-weight:700;'>entire </span>train, <span style='color:green;font-weight:700;'>no </span>but <span style='color:green;font-weight:700;'>perhaps </span>I am looking <span style='color:green;font-weight:700;'>ahead </span>for an expensive restaurant in the <span style='color:red;font-weight:700;text-decoration:line-through;'>centre </span><span style='color:green;font-weight:700;'>corner </span>of town. 

[B] We have <span style='color:green;font-weight:700;'>to </span>many expensive places in <span style='color:green;font-weight:700;'>Charlotte </span>the center of town! Is there a <span style='color:green;font-weight:700;'>pretty </span>particular type of cuisine you'd like <span style='color:red;font-weight:700;text-decoration:line-through;'>to </span><span style='color:green;font-weight:700;'>guys to finally </span>try? 

[A] I <span style='color:green;font-weight:700;'>too </span>would like modern <span style='color:green;font-weight:700;'>ized </span>European food. 

[B] How <span style='color:green;font-weight:700;'>goes </span>about <span style='color:red;font-weight:700;text-decoration:line-through;'>darrys </span><span style='color:green;font-weight:700;'>vegetarian </span>cookhouse and <span style='color:red;font-weight:700;text-decoration:line-through;'>wine </span><span style='color:green;font-weight:700;'>butcher </span>shop? 

[A] <span style='color:red;font-weight:700;text-decoration:line-through;'>Yeah </span><span style='color:green;font-weight:700;'>So </span>that <span style='color:green;font-weight:700;'>thing </span>should work can I please <span style='color:green;font-weight:700;'>you </span>get the address <span style='color:green;font-weight:700;'>plus </span>post code and <span style='color:red;font-weight:700;text-decoration:line-through;'>phone </span><span style='color:green;font-weight:700;'>subscriber </span>number? 

[B] They are <span style='color:red;font-weight:700;text-decoration:line-through;'>located </span><span style='color:green;font-weight:700;'>currently </span>at 40270 King <span style='color:red;font-weight:700;text-decoration:line-through;'>Street </span><span style='color:green;font-weight:700;'>Auckland </span>City Centre. Postcode <span style='color:green;font-weight:700;'>Card </span>is cb11ln. <span style='color:red;font-weight:700;text-decoration:line-through;'>Phone </span><span style='color:green;font-weight:700;'>Account Group </span>number <span style='color:green;font-weight:700;'>ID </span>is 01223505015. Can I book a table <span style='color:green;font-weight:700;'>arrangement </span>for you? 

[A] No <span style='color:green;font-weight:700;'>sir </span>thank you. I will <span style='color:red;font-weight:700;text-decoration:line-through;'>go </span><span style='color:green;font-weight:700;'>follow up </span>ahead and book <span style='color:green;font-weight:700;'>list </span>myself. <span style='color:green;font-weight:700;'>Okay </span>Thank you <span style='color:green;font-weight:700;'>by </span>so much. This was all the <span style='color:red;font-weight:700;text-decoration:line-through;'>info </span><span style='color:green;font-weight:700;'>training </span>I needed. Have a good day. 

[B] We appreciate <span style='color:green;font-weight:700;'>opening </span>your business. <span style='color:green;font-weight:700;'>SS </span>Thank for using the Cambridge <span style='color:red;font-weight:700;text-decoration:line-through;'>Towninfo </span><span style='color:green;font-weight:700;'>events business </span>centre! Have a great day!

### Easy Data Augmentation

Это супер быстро!

An implementation of Easy Data Augmentation, which combines:

- WordNet synonym replacement
    - Randomly replace words with their synonyms.
- Word deletion
    - Randomly remove words from the sentence.
- Word order swaps
    - Randomly swap the position of words in the sentence.
- Random synonym insertion
    - Insert a random synonym of a random word at a random location.

In [7]:
! textattack augment\
    --input-csv aug-data/original.csv\
    --output-csv aug-data/eda1.csv\
    --input-column text\
    --recipe eda\
    --pct-words-to-swap .1\
    --transformations-per-example 1\
    --exclude-original

2023-07-29 22:55:47.601277: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-29 22:55:48.305955: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-29 22:55:50.362310: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-29 22:55:50.362663: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GP

In [6]:
! textattack augment\
    --input-csv aug-data/original.csv\
    --output-csv aug-data/eda3.csv\
    --input-column text\
    --recipe eda\
    --pct-words-to-swap .3\
    --transformations-per-example 1\
    --exclude-original

2023-07-29 22:55:12.114556: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-29 22:55:12.791880: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-29 22:55:14.817800: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-29 22:55:14.818226: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GP

In [8]:
! textattack augment\
    --input-csv aug-data/original.csv\
    --output-csv aug-data/eda5.csv\
    --input-column text\
    --recipe eda\
    --pct-words-to-swap .5\
    --transformations-per-example 1\
    --exclude-original

2023-07-29 22:56:05.174414: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-29 22:56:05.932334: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-29 22:56:08.404231: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-29 22:56:08.404716: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GP

In [9]:
from visualization_utils import show_augmented


for name in ['eda1', 'eda3', 'eda5']:
    print(f'======= {name} =======')
    show_augmented(14, name)

======= eda1 =======


[A] I <span style='color:red;font-weight:700;text-decoration:line-through;'>need </span><span style='color:green;font-weight:700;'>require </span>a train from birmingham new street to cambridge, please. 

[B] There are 133 trains traveling that path. What <span style='color:green;font-weight:700;'>to would you like </span>day <span style='color:red;font-weight:700;text-decoration:line-through;'>would you like to </span>travel on? 

[A] <span style='color:red;font-weight:700;text-decoration:line-through;'>I </span><span style='color:green;font-weight:700;'>unity </span>would like to leave on Wednesday sometime after 11:30. 

[B] How about <span style='color:green;font-weight:700;'>xi </span>the TR1670? It meets your specifications and leaves at 11:40. 

[A] How long is the <span style='color:red;font-weight:700;text-decoration:line-through;'>train </span>ride? 

[B] The train ride is 163 minutes, does this <span style='color:red;font-weight:700;text-decoration:line-through;'>suit </span><span style='color:green;font-weight:700;'>needs </span>your <span style='color:red;font-weight:700;text-decoration:line-through;'>needs? </span><span style='color:green;font-weight:700;'>suit? </span>

[A] How much is the fare on that <span style='color:red;font-weight:700;text-decoration:line-through;'>train? ¶ [B] </span><span style='color:green;font-weight:700;'>civilise? </span>

<span style='color:green;font-weight:700;'>[B] IT </span>It is 75.10 pounds. 

[A] What is <span style='color:red;font-weight:700;text-decoration:line-through;'>the </span>exact arrival time? I'm also looking for <span style='color:red;font-weight:700;text-decoration:line-through;'>a </span>place to dine in the centre of town. I'm looking for an expensive modern european restaurant. 

[B] Let's take care <span style='color:red;font-weight:700;text-decoration:line-through;'>of </span><span style='color:green;font-weight:700;'>to </span>the train first. The train will arrive by 14:23 in Cambridge. Would you like <span style='color:red;font-weight:700;text-decoration:line-through;'>to </span><span style='color:green;font-weight:700;'>of </span>book a ticket? 

[A] <span style='color:red;font-weight:700;text-decoration:line-through;'>Thank </span>you I do not need to book the train, but I am looking for an expensive restaurant in <span style='color:red;font-weight:700;text-decoration:line-through;'>the </span>centre of town. 

[B] We have <span style='color:green;font-weight:700;'>stimulate </span>many expensive places in the center of town! Is there a particular type of cuisine you'd like to try? 

[A] I would like modern <span style='color:red;font-weight:700;text-decoration:line-through;'>European </span>food. 

[B] How <span style='color:red;font-weight:700;text-decoration:line-through;'>about </span><span style='color:green;font-weight:700;'>wine </span>darrys cookhouse and <span style='color:red;font-weight:700;text-decoration:line-through;'>wine </span><span style='color:green;font-weight:700;'>about </span>shop? 

[A] Yeah <span style='color:red;font-weight:700;text-decoration:line-through;'>that </span><span style='color:green;font-weight:700;'>number </span>should work can I please get the address post code and phone <span style='color:red;font-weight:700;text-decoration:line-through;'>number? </span><span style='color:green;font-weight:700;'>that? </span>

[B] They are located <span style='color:red;font-weight:700;text-decoration:line-through;'>at </span>40270 King Street City Centre. Postcode is cb11ln. Phone number is 01223505015. Can I book a table <span style='color:red;font-weight:700;text-decoration:line-through;'>for </span>you? 

[A] No thank you. I will go ahead and book myself. Thank you <span style='color:red;font-weight:700;text-decoration:line-through;'>so </span><span style='color:green;font-weight:700;'>good </span>much. This was all the info I needed. Have a <span style='color:red;font-weight:700;text-decoration:line-through;'>good </span><span style='color:green;font-weight:700;'>so </span>day. 

[B] <span style='color:red;font-weight:700;text-decoration:line-through;'>We </span><span style='color:green;font-weight:700;'>business </span>appreciate your <span style='color:red;font-weight:700;text-decoration:line-through;'>business. </span><span style='color:green;font-weight:700;'>We. </span>Thank for using the Cambridge Towninfo centre! Have a great day!

======= eda3 =======


[A] I need a <span style='color:green;font-weight:700;'>train </span>train from birmingham new street to <span style='color:green;font-weight:700;'>gearing </span>cambridge, please. 

[B] There are 133 trains <span style='color:green;font-weight:700;'>wish </span>traveling that path. What day would you like to <span style='color:green;font-weight:700;'>way </span>travel on? 

[A] <span style='color:red;font-weight:700;text-decoration:line-through;'>I </span><span style='color:green;font-weight:700;'>after </span>would like to leave <span style='color:red;font-weight:700;text-decoration:line-through;'>on </span><span style='color:green;font-weight:700;'>I </span>Wednesday sometime <span style='color:red;font-weight:700;text-decoration:line-through;'>after </span><span style='color:green;font-weight:700;'>on </span>11:30. 

[B] How about the TR1670? It meets <span style='color:green;font-weight:700;'>eleven </span>your specifications and leaves at <span style='color:green;font-weight:700;'>spec </span>11:40. 

[A] How long is <span style='color:red;font-weight:700;text-decoration:line-through;'>the </span>train ride? 

[B] <span style='color:green;font-weight:700;'>163 train needs is </span>The <span style='color:red;font-weight:700;text-decoration:line-through;'>train ride is 163 </span>minutes, does this suit your <span style='color:red;font-weight:700;text-decoration:line-through;'>needs? </span><span style='color:green;font-weight:700;'>ride? </span>

[A] How much <span style='color:green;font-weight:700;'>be </span>is the fare on that train? 

[B] It <span style='color:red;font-weight:700;text-decoration:line-through;'>is 75.10 </span><span style='color:green;font-weight:700;'>10 75.is </span>pounds. 

[A] What is the exact <span style='color:red;font-weight:700;text-decoration:line-through;'>arrival time? </span><span style='color:green;font-weight:700;'>comer metre? </span>I'm also looking for a <span style='color:red;font-weight:700;text-decoration:line-through;'>place </span><span style='color:green;font-weight:700;'>direct </span>to dine in the <span style='color:red;font-weight:700;text-decoration:line-through;'>centre of town. </span><span style='color:green;font-weight:700;'>marrow of township. </span>I'm <span style='color:red;font-weight:700;text-decoration:line-through;'>looking </span><span style='color:green;font-weight:700;'>expect </span>for an expensive <span style='color:red;font-weight:700;text-decoration:line-through;'>modern european </span><span style='color:green;font-weight:700;'>advanced European </span>restaurant. 

[B] <span style='color:green;font-weight:700;'>same </span>Let's take <span style='color:green;font-weight:700;'>xxiii </span>care of the train <span style='color:green;font-weight:700;'>beginning </span>first. The train will arrive by 14:23 in Cambridge. Would you like to book a ticket? 

[A] Thank <span style='color:green;font-weight:700;'>Town </span>you I do <span style='color:green;font-weight:700;'>be </span>not need to book the train, but I am looking for an expensive restaurant <span style='color:green;font-weight:700;'>prepare </span>in the centre of town. 

[B] We have <span style='color:green;font-weight:700;'>get </span>many expensive places in the center <span style='color:green;font-weight:700;'>exceptional </span>of town! Is there a particular type of cuisine <span style='color:green;font-weight:700;'>engender </span>you'd like to try? 

[A] I would like <span style='color:red;font-weight:700;text-decoration:line-through;'>modern </span><span style='color:green;font-weight:700;'>Bodoni </span>European food. 

[B] How about darrys cookhouse and <span style='color:red;font-weight:700;text-decoration:line-through;'>wine shop? </span><span style='color:green;font-weight:700;'>vino patronise? </span>

[A] Yeah that should <span style='color:green;font-weight:700;'>fetch </span>work can <span style='color:green;font-weight:700;'>delight </span>I please get the address post code and phone number? 

[B] <span style='color:red;font-weight:700;text-decoration:line-through;'>They </span><span style='color:green;font-weight:700;'>number </span>are <span style='color:red;font-weight:700;text-decoration:line-through;'>located at </span><span style='color:green;font-weight:700;'>01223505015 I </span>40270 King Street City Centre. Postcode is cb11ln. Phone <span style='color:red;font-weight:700;text-decoration:line-through;'>number is 01223505015. </span><span style='color:green;font-weight:700;'>They is located. </span>Can <span style='color:red;font-weight:700;text-decoration:line-through;'>I </span><span style='color:green;font-weight:700;'>at </span>book a table for you? 

[A] No thank you. I will <span style='color:red;font-weight:700;text-decoration:line-through;'>go ahead </span><span style='color:green;font-weight:700;'>plump beforehand </span>and <span style='color:red;font-weight:700;text-decoration:line-through;'>book </span><span style='color:green;font-weight:700;'>Koran </span>myself. Thank you so much. This was all the info <span style='color:red;font-weight:700;text-decoration:line-through;'>I needed. Have a good </span><span style='color:green;font-weight:700;'>unity requisite. bear a just </span>day. 

[B] We <span style='color:red;font-weight:700;text-decoration:line-through;'>appreciate </span><span style='color:green;font-weight:700;'>treasure </span>your business. Thank for <span style='color:red;font-weight:700;text-decoration:line-through;'>using </span><span style='color:green;font-weight:700;'>victimisation </span>the Cambridge Towninfo <span style='color:red;font-weight:700;text-decoration:line-through;'>centre! </span><span style='color:green;font-weight:700;'>eye! </span>Have a <span style='color:red;font-weight:700;text-decoration:line-through;'>great </span><span style='color:green;font-weight:700;'>bang-up </span>day!

======= eda5 =======


[A] <span style='color:red;font-weight:700;text-decoration:line-through;'>I need a train </span><span style='color:green;font-weight:700;'>iodin demand a develop </span>from birmingham <span style='color:red;font-weight:700;text-decoration:line-through;'>new </span><span style='color:green;font-weight:700;'>freshly </span>street to <span style='color:red;font-weight:700;text-decoration:line-through;'>cambridge, </span><span style='color:green;font-weight:700;'>Cambridge, </span>please. 

[B] <span style='color:red;font-weight:700;text-decoration:line-through;'>There are 133 </span><span style='color:green;font-weight:700;'>would that you </span>trains traveling <span style='color:red;font-weight:700;text-decoration:line-through;'>that </span><span style='color:green;font-weight:700;'>are </span>path. <span style='color:red;font-weight:700;text-decoration:line-through;'>What </span><span style='color:green;font-weight:700;'>on </span>day <span style='color:green;font-weight:700;'>There 133 like to travel What? </span>

<span style='color:green;font-weight:700;'>[A] iodine </span>would <span style='color:red;font-weight:700;text-decoration:line-through;'>you like to travel on? ¶ [A] I would like to leave </span><span style='color:green;font-weight:700;'>similar to leave-taking </span>on Wednesday <span style='color:red;font-weight:700;text-decoration:line-through;'>sometime </span><span style='color:green;font-weight:700;'>former </span>after <span style='color:red;font-weight:700;text-decoration:line-through;'>11:30. ¶ [B] How </span><span style='color:green;font-weight:700;'>xi:30. </span>

<span style='color:green;font-weight:700;'>[B] How astatine </span>about the <span style='color:green;font-weight:700;'>spec </span>TR1670? It <span style='color:green;font-weight:700;'>IT </span>meets your specifications and leaves at 11:40. 

[A] How <span style='color:green;font-weight:700;'>hanker </span>long is <span style='color:green;font-weight:700;'>personify </span>the train ride? 

[B] The train ride <span style='color:red;font-weight:700;text-decoration:line-through;'>is </span>163 minutes, <span style='color:red;font-weight:700;text-decoration:line-through;'>does this suit your needs? ¶ [A] How much is the fare </span><span style='color:green;font-weight:700;'>suit? </span>

<span style='color:green;font-weight:700;'>[A] How lots is the do </span>on that <span style='color:red;font-weight:700;text-decoration:line-through;'>train? </span><span style='color:green;font-weight:700;'>coach? </span>

[B] It is <span style='color:red;font-weight:700;text-decoration:line-through;'>75.10 pounds. </span><span style='color:green;font-weight:700;'>75.tenner punt. </span>

[A] What is the <span style='color:red;font-weight:700;text-decoration:line-through;'>exact arrival time? </span><span style='color:green;font-weight:700;'>accurate comer meter? </span>I'm <span style='color:red;font-weight:700;text-decoration:line-through;'>also looking for a place </span><span style='color:green;font-weight:700;'>besides reckon for a identify </span>to dine in the <span style='color:red;font-weight:700;text-decoration:line-through;'>centre of town. </span><span style='color:green;font-weight:700;'>Centre of townsfolk. </span>I'm <span style='color:red;font-weight:700;text-decoration:line-through;'>looking </span><span style='color:green;font-weight:700;'>bet </span>for an expensive <span style='color:red;font-weight:700;text-decoration:line-through;'>modern european restaurant. </span><span style='color:green;font-weight:700;'>innovative European eatery. </span>

[B] Let's <span style='color:red;font-weight:700;text-decoration:line-through;'>take care of the train first. </span><span style='color:green;font-weight:700;'>assume deal of the educate initiative. </span>The <span style='color:red;font-weight:700;text-decoration:line-through;'>train </span><span style='color:green;font-weight:700;'>check </span>will <span style='color:red;font-weight:700;text-decoration:line-through;'>arrive </span><span style='color:green;font-weight:700;'>come </span>by <span style='color:red;font-weight:700;text-decoration:line-through;'>14:23 </span><span style='color:green;font-weight:700;'>XIV:twenty-three </span>in Cambridge. Would you <span style='color:red;font-weight:700;text-decoration:line-through;'>like to book a ticket? ¶ [A] Thank you I </span><span style='color:green;font-weight:700;'>similar to Bible a tag? </span>

<span style='color:green;font-weight:700;'>[A] thank you ace </span>do not <span style='color:red;font-weight:700;text-decoration:line-through;'>need to book the train, </span><span style='color:green;font-weight:700;'>require to Bible the cultivate, </span>but <span style='color:red;font-weight:700;text-decoration:line-through;'>I </span><span style='color:green;font-weight:700;'>i </span>am <span style='color:red;font-weight:700;text-decoration:line-through;'>looking </span><span style='color:green;font-weight:700;'>look </span>for an expensive <span style='color:red;font-weight:700;text-decoration:line-through;'>restaurant in the centre of town. </span><span style='color:green;font-weight:700;'>eatery in the kernel of township. </span>

[B] We have many expensive places in the center of town! Is there a <span style='color:green;font-weight:700;'>township </span>particular <span style='color:green;font-weight:700;'>indium </span>type <span style='color:green;font-weight:700;'>pose </span>of cuisine you'd <span style='color:green;font-weight:700;'>be </span>like <span style='color:green;font-weight:700;'>town </span>to try? 

[A] I <span style='color:green;font-weight:700;'>European food modern </span>would <span style='color:red;font-weight:700;text-decoration:line-through;'>like modern European food. ¶ [B] How </span><span style='color:green;font-weight:700;'>like. </span>

<span style='color:green;font-weight:700;'>[B] wine </span>about darrys cookhouse <span style='color:green;font-weight:700;'>How </span>and <span style='color:red;font-weight:700;text-decoration:line-through;'>wine </span>shop? 

[A] Yeah <span style='color:red;font-weight:700;text-decoration:line-through;'>that </span>should <span style='color:red;font-weight:700;text-decoration:line-through;'>work can I </span>please get the address <span style='color:red;font-weight:700;text-decoration:line-through;'>post </span>code <span style='color:red;font-weight:700;text-decoration:line-through;'>and phone </span>number? 

[B] They are <span style='color:red;font-weight:700;text-decoration:line-through;'>located </span><span style='color:green;font-weight:700;'>situated </span>at 40270 <span style='color:red;font-weight:700;text-decoration:line-through;'>King Street </span><span style='color:green;font-weight:700;'>magnate street </span>City <span style='color:red;font-weight:700;text-decoration:line-through;'>Centre. Postcode </span><span style='color:green;font-weight:700;'>center. postcode </span>is cb11ln. <span style='color:red;font-weight:700;text-decoration:line-through;'>Phone number </span><span style='color:green;font-weight:700;'>phone figure </span>is 01223505015. <span style='color:red;font-weight:700;text-decoration:line-through;'>Can I book a table </span><span style='color:green;font-weight:700;'>stool 1 al-Qur'an a mesa </span>for you? 

[A] <span style='color:red;font-weight:700;text-decoration:line-through;'>No </span><span style='color:green;font-weight:700;'>nobelium </span>thank you. <span style='color:red;font-weight:700;text-decoration:line-through;'>I </span><span style='color:green;font-weight:700;'>ane </span>will <span style='color:red;font-weight:700;text-decoration:line-through;'>go ahead </span><span style='color:green;font-weight:700;'>function onward </span>and <span style='color:red;font-weight:700;text-decoration:line-through;'>book </span><span style='color:green;font-weight:700;'>Scripture </span>myself. <span style='color:red;font-weight:700;text-decoration:line-through;'>Thank </span><span style='color:green;font-weight:700;'>thank </span>you so much. This was all the <span style='color:red;font-weight:700;text-decoration:line-through;'>info I needed. Have a good day. </span><span style='color:green;font-weight:700;'>information 1 required. take a secure Day. </span>

[B] We appreciate your business. Thank <span style='color:green;font-weight:700;'>daytime </span>for using the <span style='color:green;font-weight:700;'>daylight </span>Cambridge Towninfo <span style='color:green;font-weight:700;'>occupation </span>centre! Have a <span style='color:green;font-weight:700;'>apprize </span>great day!

### Back Translation

In [5]:
! textattack augment\
    --input-csv aug-data/original.csv\
    --output-csv aug-data/back_trans1.csv\
    --input-column text\
    --recipe back_trans\
    --pct-words-to-swap .1\
    --transformations-per-example 1\
    --exclude-original

2023-07-30 00:29:06.196491: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-30 00:29:06.765555: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-30 00:29:08.530649: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-30 00:29:08.530955: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GP

## `googletrans` package

Это неофициальное API

In [14]:
from googletrans import Translator
from visualization_utils import read_csv

original = read_csv('aug-data/original.csv')
translator = Translator(['translate.google.ru'])
ru = translator.translate(original[0], src='en', dest='ru').text
aug = translator.translate(ru, src='ru', dest='en')

AttributeError: 'NoneType' object has no attribute 'group'

## Hugging Face Back Translation

In [1]:
from my_augmentations import back_translate


back_translate()

2023-07-30 14:48:16.004098: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-30 14:48:16.909017: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
No model was supplied, defaulted to t5-base and revision 686f1db (https://huggingface.co/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
/home/ilya/.local/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware tha

In [2]:
from transformers import pipeline
from visualization_utils import read_csv


# to french
translator = pipeline('translation_en_to_fr', model='Helsinki-NLP/opus-mt-en-fr', device='cuda:0')
original = read_csv('aug-data/original.csv')
translated = [a['translation_text'] for a in translator(original)]
del translator

In [3]:
# back to english
translator = pipeline('translation_fr_to_en', model='Helsinki-NLP/opus-mt-fr-en', device='cuda:0')
back_translated = [a['translation_text'] for a in translator(translated)]
del translator

In [5]:
import pandas as pd


# save to csv
df = pd.DataFrame({'text': back_translated})
df.to_csv(f'aug-data/back_translated.csv')

## Insertion

- Цель: добавить слова подхоящие в контекст.
- Идея: добавить маскированные токены и предсказать их с помощью XLNet.
- Использовать XLNet чтобы обрабатывать весь диалог сразу, а не отдельно по одному уттерансу

In [1]:
from my_augmentations import Inserter


inserter = Inserter(
    fraction=0.1,
    score_threshold=0.005,
    k=3,
    mask_utterance_level=False,
    fill_utterance_level=True,
    model='xlnet-base-cased',
    device='cuda'
)
inserter.from_file_system('insertion')

2023-08-02 03:10:07.834982: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-02 03:10:08.446552: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package stopwords to /home/ilya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from my_augmentations import Inserter


inserter = Inserter(
    fraction=0.1,
    score_threshold=0.005,
    k=5,
    mask_utterance_level=False,
    fill_utterance_level=False,
    model='xlnet-base-cased',
    device='cuda'
)
inserter.from_file_system('xlnet-insertion')

[nltk_data] Downloading package stopwords to /home/ilya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Replacement

In [1]:
from my_augmentations import Replacer


replacer = Replacer(
    k=3,
    fill_utterance_level=3,
    model='microsoft/mpnet-base',
    device='cuda'
)
replacer.from_file_system('replacement3')

2023-08-01 20:59:52.999292: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-01 20:59:53.610733: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package stopwords to /home/ilya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/ilya/.local/lib/python3.10/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


- визуально оценить результат в зависимости от level

## ChatGPT API

In [72]:
from dotenv import load_dotenv

load_dotenv()

True

In [73]:
import os
import openai


openai.api_key = os.environ['OPENAI_KEY']

In [74]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [75]:
prompt = """
Below, in triple backticks, is a dialogue between a client and a service worker on the topic of {topic}. The symbols [A] and [B] indicate the belonging of the remarks to the client and the employee (which of them specifically is unknown).

Your task is to rewrite the original dialogue in an informal style. You can change the vocabulary. You can not change the order of the lines, change the meaning of the dialogue and phrases in general.

Respond only with dialogue, without any additional comments.

```{dialogue}```
"""

In [77]:
responses = []
for i, (topic, dialogue) in enumerate(zip(topics, dialogues)):
    if i > 10:
        break
    response = get_completion(prompt.format(topic=', '.join(topic), dialogue=dialogue))
    responses.append(response)

RateLimitError: You exceeded your current quota, please check your plan and billing details.

In [78]:
responses

[]

## ChatGPT Web UI

In [69]:
from redlines import Redlines
from IPython.display import Markdown, display


def test(orig, new):
    test = Redlines(orig, new)
    display(Markdown(test.output_markdown))

### Stop Utterances

Промт такой:
```
Below, in triple quotes, is a dialogue between a client and a service worker on the topic of hotel, taxi and entertainment. The symbols [A] and [B] indicate the belonging of the replicas to the client and the employee (which of them specifically is unknown).

Your task is to add to some replicas such phrases that do not add or distort the meaning of the dialogue as a whole, do not change the sequence of replies of the speakers. Added phrases should be no longer than 5-10 words. You can add to any part of the dialogue and to any part of phrases.

You are not allowed to delete words and phrases or change the words of the original dialogue in any way. The final dialogue must comply with the rules of grammar.

Are you ready to get dialogue in triple quotes? Say yes or no
```

In [49]:
orig = """
[A] My husband and I are celebrating our anniversary and want to find a great place to stay in town.
[B] Congratulations on your upcoming anniversary! Cambridge offers a variety of lodging options, what is your price range?
[A] I would like a 4 star guesthouse that includes free parking.
[B] I have several options for you, is there a particular area you are interested in during your stay?
[A] yes should be in the west
[B] I have one guesthouse that fits that criteria, Finches Bed and Breakfast. Would you like me to book for you?
[A] Yes, please! We'll arrive on Monday and stay 2 nights. Just the two of us, of course!
[B] Ok, your hotel stay at Finches Bed and Breakfast is booked, Reference number FKRO2HOW . Will there be anything else?
[A] I am wanting to know more about the Cambridge Museum of Technology.
[B] Sure, it's located in the centre area of town. The phone number is 01223368650. The entrance fee is 5 pounds.
[A] What is the address, please.
[B] The address is the old pumping station, cheddars lane. Did you want any more information about the museum?
[A] Yes, I need the entrance fee. I will also need a taxi to pick me up by 24:30. I need the contact number and car type please.
[B] The entrance fee is 5 pounds.
[A] Can you book the taxi for me at 24:30 leaving from the museum please. I will need the contact number and the car type.
[B] and what is your destination ?
[A] I will leave the Cambridge Museum at 24:30, going to the the Finches Bed and Breakfast.
[B] Your taxi has been booked. It will be a grey volvo and the contact number is 07239297218.
[A] Thanks so much. That is all I need for today. Bye.
[B] Have a nice stay!
"""

In [50]:
new = """
<A> My husband and I are celebrating our anniversary and want to find a great place to stay in town.
<B> Congratulations on your upcoming anniversary! Cambridge offers a variety of lodging options, what is your price range?
<A> I would like a 4-star guesthouse that includes free parking, please.
<B> I have several options for you. Is there a particular area you are interested in during your stay?
<A> Yes, it should be in the west, please.
<B> I have one guesthouse that fits that criteria, Finches Bed and Breakfast. Would you like me to book it for you?
<A> Yes, please! We'll arrive on Monday and stay 2 nights. Just the two of us, of course!
<B> Ok, your hotel stay at Finches Bed and Breakfast is booked, Reference number FKRO2HOW. Will there be anything else?
<A> I am wanting to know more about the Cambridge Museum of Technology, actually.
<B> Sure, it's located in the center area of town. The phone number is 01223368650. The entrance fee is 5 pounds.
<A> What is the address, please.
<B> The address is the old pumping station, Cheddars Lane. Did you want any more information about the museum?
<A> Yes, I need the entrance fee. Also, I will need a taxi to pick me up by 24:30. Can you provide the contact number and car type, please?
<B> The entrance fee is 5 pounds.
<A> Can you book the taxi for me at 24:30, leaving from the museum, please? I will need the contact number and the car type.
<B> Sure, and what is your destination?
<A> I will leave the Cambridge Museum at 24:30, going to Finches Bed and Breakfast.
<B> Your taxi has been booked. It will be a grey Volvo, and the contact number is 07239297218.
<A> Thanks so much. That is all I need for today. Bye.
<B> Have a nice stay!
"""

In [55]:
def repl(txt):
    txt = txt.replace('<', '[')
    return txt.replace('>', ']')

In [56]:
new = repl(new)

In [61]:
test(orig, new)

[A] My husband and I are celebrating our anniversary and want to find a great place to stay in town. 

[B] Congratulations on your upcoming anniversary! Cambridge offers a variety of lodging options, what is your price range? 

[A] I would like a <span style='color:red;font-weight:700;text-decoration:line-through;'>4 star </span><span style='color:green;font-weight:700;'>4-star </span>guesthouse that includes free <span style='color:red;font-weight:700;text-decoration:line-through;'>parking. </span><span style='color:green;font-weight:700;'>parking, please. </span>

[B] I have several options for <span style='color:red;font-weight:700;text-decoration:line-through;'>you, is </span><span style='color:green;font-weight:700;'>you. Is </span>there a particular area you are interested in during your stay? 

[A] <span style='color:red;font-weight:700;text-decoration:line-through;'>yes </span><span style='color:green;font-weight:700;'>Yes, it </span>should be in the <span style='color:red;font-weight:700;text-decoration:line-through;'>west </span><span style='color:green;font-weight:700;'>west, please. </span>

[B] I have one guesthouse that fits that criteria, Finches Bed and Breakfast. Would you like me to book <span style='color:green;font-weight:700;'>it </span>for you? 

[A] Yes, please! We'll arrive on Monday and stay 2 nights. Just the two of us, of course! 

[B] Ok, your hotel stay at Finches Bed and Breakfast is booked, Reference number <span style='color:red;font-weight:700;text-decoration:line-through;'>FKRO2HOW . </span><span style='color:green;font-weight:700;'>FKRO2HOW. </span>Will there be anything else? 

[A] I am wanting to know more about the Cambridge Museum of <span style='color:red;font-weight:700;text-decoration:line-through;'>Technology. </span><span style='color:green;font-weight:700;'>Technology, actually. </span>

[B] Sure, it's located in the <span style='color:red;font-weight:700;text-decoration:line-through;'>centre </span><span style='color:green;font-weight:700;'>center </span>area of town. The phone number is 01223368650. The entrance fee is 5 pounds. 

[A] What is the address, please. 

[B] The address is the old pumping station, <span style='color:red;font-weight:700;text-decoration:line-through;'>cheddars lane. </span><span style='color:green;font-weight:700;'>Cheddars Lane. </span>Did you want any more information about the museum? 

[A] Yes, I need the entrance fee. <span style='color:green;font-weight:700;'>Also, </span>I will <span style='color:red;font-weight:700;text-decoration:line-through;'>also </span>need a taxi to pick me up by 24:30. <span style='color:red;font-weight:700;text-decoration:line-through;'>I need </span><span style='color:green;font-weight:700;'>Can you provide </span>the contact number and car <span style='color:red;font-weight:700;text-decoration:line-through;'>type please. </span><span style='color:green;font-weight:700;'>type, please? </span>

[B] The entrance fee is 5 pounds. 

[A] Can you book the taxi for me at <span style='color:red;font-weight:700;text-decoration:line-through;'>24:30 </span><span style='color:green;font-weight:700;'>24:30, </span>leaving from the <span style='color:red;font-weight:700;text-decoration:line-through;'>museum please. </span><span style='color:green;font-weight:700;'>museum, please? </span>I will need the contact number and the car type. 

[B] <span style='color:green;font-weight:700;'>Sure, </span>and what is your <span style='color:red;font-weight:700;text-decoration:line-through;'>destination ? </span><span style='color:green;font-weight:700;'>destination? </span>

[A] I will leave the Cambridge Museum at 24:30, going to <span style='color:red;font-weight:700;text-decoration:line-through;'>the the </span>Finches Bed and Breakfast. 

[B] Your taxi has been booked. It will be a grey <span style='color:red;font-weight:700;text-decoration:line-through;'>volvo </span><span style='color:green;font-weight:700;'>Volvo, </span>and the contact number is 07239297218. 

[A] Thanks so much. That is all I need for today. Bye. 

[B] Have a nice stay!

In [63]:
new2 = """
<A> My husband and I are celebrating our anniversary and want to find a great place to stay in town.
<B> Congratulations on your upcoming anniversary! Cambridge offers a variety of lodging options, what is your price range?
<A> I would like a 4-star guesthouse that includes free parking, please.
<B> I have several options for you. Is there a particular area in the west you are interested in during your stay?
<A> Yes, it should be in the west.
<B> I have one guesthouse that fits that criteria, Finches Bed and Breakfast. Would you like me to book it for you?
<A> Yes, please! We'll arrive on Monday and stay 2 nights, just the two of us, of course!
<B> Ok, your hotel stay at Finches Bed and Breakfast is booked. Here is the reference number FKRO2HOW. Will there be anything else?
<A> I am wanting to know more about the Cambridge Museum of Technology.
<B> Sure, it's located in the center area of town. The phone number is 01223368650. The entrance fee is 5 pounds.
<A> What is the address, please?
<B> The address is the old pumping station, Cheddars Lane. Did you want any more information about the museum?
<A> Yes, I need to know the entrance fee. I will also need a taxi to pick me up by 24:30. Could you provide the contact number and car type, please?
<B> The entrance fee is 5 pounds.
<A> Can you please book the taxi for me at 24:30, leaving from the museum? I will need the contact number and the car type.
<B> Sure, your taxi has been booked. It will be a grey Volvo, and the contact number is 07239297218.
<A> Thanks so much. That is all I need for today. Bye.
<B> Have a nice stay!
"""

In [64]:
test(orig, new2)

[A] My husband and I are celebrating our anniversary and want to find a great place to stay in town. 

[B] Congratulations on your upcoming anniversary! Cambridge offers a variety of lodging options, what is your price range? 

[A] I would like a <span style='color:red;font-weight:700;text-decoration:line-through;'>4 star </span><span style='color:green;font-weight:700;'>4-star </span>guesthouse that includes free <span style='color:red;font-weight:700;text-decoration:line-through;'>parking. </span><span style='color:green;font-weight:700;'>parking, please. </span>

[B] I have several options for <span style='color:red;font-weight:700;text-decoration:line-through;'>you, is </span><span style='color:green;font-weight:700;'>you. Is </span>there a particular area <span style='color:green;font-weight:700;'>in the west </span>you are interested in during your stay? 

[A] <span style='color:red;font-weight:700;text-decoration:line-through;'>yes </span><span style='color:green;font-weight:700;'>Yes, it </span>should be in the <span style='color:red;font-weight:700;text-decoration:line-through;'>west </span><span style='color:green;font-weight:700;'>west. </span>

[B] I have one guesthouse that fits that criteria, Finches Bed and Breakfast. Would you like me to book <span style='color:green;font-weight:700;'>it </span>for you? 

[A] Yes, please! We'll arrive on Monday and stay 2 <span style='color:red;font-weight:700;text-decoration:line-through;'>nights. Just </span><span style='color:green;font-weight:700;'>nights, just </span>the two of us, of course! 

[B] Ok, your hotel stay at Finches Bed and Breakfast is <span style='color:red;font-weight:700;text-decoration:line-through;'>booked, Reference number FKRO2HOW . </span><span style='color:green;font-weight:700;'>booked. Here is the reference number FKRO2HOW. </span>Will there be anything else? 

[A] I am wanting to know more about the Cambridge Museum of Technology. 

[B] Sure, it's located in the <span style='color:red;font-weight:700;text-decoration:line-through;'>centre </span><span style='color:green;font-weight:700;'>center </span>area of town. The phone number is 01223368650. The entrance fee is 5 pounds. 

[A] What is the address, <span style='color:red;font-weight:700;text-decoration:line-through;'>please. </span><span style='color:green;font-weight:700;'>please? </span>

[B] The address is the old pumping station, <span style='color:red;font-weight:700;text-decoration:line-through;'>cheddars lane. </span><span style='color:green;font-weight:700;'>Cheddars Lane. </span>Did you want any more information about the museum? 

[A] Yes, I need <span style='color:green;font-weight:700;'>to know </span>the entrance fee. I will also need a taxi to pick me up by 24:30. <span style='color:red;font-weight:700;text-decoration:line-through;'>I need </span><span style='color:green;font-weight:700;'>Could you provide </span>the contact number and car <span style='color:red;font-weight:700;text-decoration:line-through;'>type please. </span><span style='color:green;font-weight:700;'>type, please? </span>

[B] The entrance fee is 5 pounds. 

[A] Can you <span style='color:green;font-weight:700;'>please </span>book the taxi for me at <span style='color:red;font-weight:700;text-decoration:line-through;'>24:30 </span><span style='color:green;font-weight:700;'>24:30, </span>leaving from the <span style='color:red;font-weight:700;text-decoration:line-through;'>museum please. </span><span style='color:green;font-weight:700;'>museum? </span>I will need the contact number and the car type. 

[B] <span style='color:red;font-weight:700;text-decoration:line-through;'>and what is your destination ? ¶ [A] I will leave the Cambridge Museum at 24:30, going to the the Finches Bed and Breakfast. ¶ [B] Your </span><span style='color:green;font-weight:700;'>Sure, your </span>taxi has been booked. It will be a grey <span style='color:red;font-weight:700;text-decoration:line-through;'>volvo </span><span style='color:green;font-weight:700;'>Volvo, </span>and the contact number is 07239297218. 

[A] Thanks so much. That is all I need for today. Bye. 

[B] Have a nice stay!

### Informal Style

Промт:
```
Below, in triple quotes, is a dialogue between a client and a service worker on the topic of a hotel, taxi and entertainment. The symbols [A] and [B] indicate the belonging of the remarks to the client and the employee (which of them specifically is unknown).

Your task is to rewrite the original dialogue in an informal style. You can change the vocabulary. You can not change the order of the lines, change the meaning of the dialogue and phrases in general.

Are you ready to get the dialogue in triple quotes? Say yes or no
```

In [ ]:
new = """
[A] Hey, my hubby and I are gonna celebrate our anniversary and we're on the hunt for an awesome place to crash in town.
[B] Oh, congrats on your upcoming anniversary! Cambridge has lots of places to stay. What's your budget like?
[A] I'm thinking a 4-star guesthouse with free parking.
[B] Gotcha! Any specific area you wanna be in during your stay?
[A] Yeah, somewhere in the west would be perfect.
[B] Cool, I've got one option that fits the bill: Finches Bed and Breakfast. Wanna book it?
[A] Absolutely! We'll be rolling in on Monday and staying for 2 nights. Just us two, ya know!
[B] Awesome! I've booked your stay at Finches Bed and Breakfast. Your reference number is FKRO2HOW. Need anything else?
[A] Yeah, I wanna know more about the Cambridge Museum of Technology.
[B] No problemo! It's in the center of town. You can reach them at 01223368650, and the entrance fee is 5 quid.
[A] Sweet, thanks! And where exactly is it located?
[B] The address is the old pumping station, Cheddars Lane. Need more info about the museum?
[A] Yeah, I need the entrance fee again. Oh, and I'll also need a cab to pick us up at 24:30. Can you give me the contact number and car type, please?
[B] Entrance fee is 5 quid, just to remind ya. And sure thing, I've booked the taxi for ya. It'll be a grey Volvo, and the contact number is 07239297218.
[A] Thanks a bunch! That's all I needed for today. Bye!
[B] Have a blast during your stay! Enjoy!
"""